# Инструмент для обработки текста и подсчета слов

#### Привет! :) У тебя есть файл с текстом и тебе нужно посчитать в нем упоминания слов по частям речи? Ты обратился по адресу! 

Инструмент состоит из нескольких функций с возможностью настройки (ниже оглавление с гиперссылками на часть файла):

* [Считывание и запись файла формата txt или excel в переменную](#Считывание-файла)


* [Очистка текста от знаков препинания, приведение к нижнему регистру](#Очистка-текста)


* [Нормализация/лемматизация слов (приведение к начальной форме)](#Нормализация/лемматизация-слов)


* [Подсчет слов и определение частей речи](#Подсчет-слов-по-частям-речи)


* [Запись всех частей речи или только нужных тебе в таблицу excel](#Фильтрация-и-запись-в-таблицу)


* [Справочный блок](#Установка-библиотек)

Если в процессе работы у тебя возникнут предложения по улучшению инструмента, можно написать мне в тг: @esthesuntik

*Примечание: код подразумевает, что все библиотеки для работы уже установлены. Справочный блок с информацией про то, как устанавливать необходимые пакеты находится в конце инструмента. Запускай его, только если впервые используешь ту или иную билиотеку*

### Считывание файла

Сначала импортируем библиотеки для этого блока

In [ ]:
#для работы с табличными данными

import pandas as pd 


#для считывания файла формата .docx

import docx


#для устранения ошибок кодировки при чтении .txt

import codecs

! Файл с текстом, который ты собираешься обрабатывать, должен лежать в той же папке, что и файл инструмента (т.к. мы не прописываем точный путь к файлу в памяти устройства) !

In [ ]:
#впиши в кавычки название файла в формате: название.xlsx (.txt / .docx)

file_name = 'file_name.docx'  

In [ ]:
#если тексты записаны в таблицу, впиши в переменную ниже название столбца, в котором они содержатся

texts_column = 'Название_столбца'

In [ ]:
#список с возможными формата файла для обработки

possible_formats = ['xlsx', 'txt', 'docx']

In [ ]:
if '.' not in file_name or file_name.split('.')[1] not in possible_formats:   #если что-то пошло не так
    print('Упс, похоже что-то пошло не так. Алгоритм для такого формата не прописан или есть ошибка в названии файла :(')

else:
    n = 0
    
    if file_name.split('.')[1] == 'xlsx':        #если формат файла xlsx, то:
        text_df = pd.read_excel(file_name)       #считываем и преобразуем таблицу в датафрейм
        texts_list = list(text_df[texts_column]) #собираем тексты из ячеек в список строк
        pre_text = ' '.join(texts_list)          #объединяем в одну строку через пробел
        
        n+=1

    elif file_name.split('.')[1] == 'txt':                 #если формат файла txt, то:
        txt_file = codecs.open(file_name, 'r', 'utf-8')    #открываем файл и записываем в переменную в формате строки

        pre_text = ''
        for line in txt_file:
            pre_text += line.strip() + ' '

        txt_file.close()
        
        n+=1

    elif file_name.split('.')[1] == 'docx':      #если формат файла docx, то:
        doc_file = docx.Document(file_name)      #считываем файл и записываем в переменную в формате строки

        pre_text = ''
        for par in doc_file.paragraphs:
            pre_text += par.text
            
        n+=1
        
    if n == 1:
        print('Ура! Текст успешно обработан и записан в переменную "pre_text" в формате строки')

### Очистка текста

Импортируем библиотеки

In [ ]:
from nltk.corpus import stopwords             #стоп-слова (некоторые предлоги, местоимения и т.д.)
ru_stopwords = stopwords.words('russian')

from string import punctuation                #знаки препинания и символы (ниже через плюс можно добавить свои)
punctuation = punctuation + '—' + '«' + '»' + '–' + '…' + '№' + '0123456789'

from emoji import is_emoji                    #определение эмодзи

In [ ]:
print(punctuation)   #от каких знаков препинания и символов избавляемся

In [ ]:
print(ru_stopwords)   #от каких стоп-слов избавляемся

In [ ]:
#если не хочешь избавляться от стоп-слов, поставь значение переменной ниже False (без кавычек)

#хочешь убрать стоп-слова, ставь значение True (без кавычек)

no_stopwords = False

Функция очистки текста

In [ ]:
def text_cleaner(words_status, text):
    
    new_text = ''
    
    for part in text.split('\n'):         #убираем перенос строки, если он есть
        if part != '':
            new_text += part
            
    no_punct_text = ''
    
    for symbol in new_text:
        if symbol not in punctuation and not is_emoji(symbol):        #удаляем знаки препинания и символы
            no_punct_text += symbol.lower()
        if symbol == '.':
            no_punct_text += ' '
            
    while '  ' in no_punct_text:                          #убираем двойные пробелы
        no_punct_text = no_punct_text.replace('  ', ' ')     
    
    if words_status == True:       #если нужно, удаляем стоп-слова
        return ' '.join([word for word in no_punct_text.split() if word not in ru_stopwords])
    
    elif words_status == False:    #если не нужно, то возвращаем просто текст без знаков препинания
        return no_punct_text

In [ ]:
clean_text = text_cleaner(no_stopwords, pre_text)

### Нормализация/лемматизация слов

Импортируем библиотеки

In [ ]:
from pymorphy3 import MorphAnalyzer           #для приведения слов к начальной форме
pymorphy3_analyzer = MorphAnalyzer()

Функция лемматизации

In [ ]:
def text_lemmatizer(text):
    res = [pymorphy3_analyzer.parse(word)[0].normal_form for word in text.split()]
    return ' '.join(res)

In [ ]:
normal_text = text_lemmatizer(clean_text)

### Подсчет слов по частям речи

Ниже можно подсчитать два вида частоты слова в тексте:
- абсолютную: число повторений слова в тексте
- относительную: доля числа употреблений слова от общего числа токенов (единиц текста, т.е. неуникальных слов)

Импортируем библиотеки

In [ ]:
from collections import Counter   #упрощает подсчет повтора слов

Считаем абсолютную и относительную частоту

In [ ]:
word_count = Counter()

words = []
count_abs = []   #список для значений абсолютной частоты
count_rel = []   #список для значений относительной частоты

normal_list = normal_text.split()
tokens_count = len(normal_list)
    
for word in normal_list:
    word_count[word] += 1        #формируем словарь с числом повторений слов              

n = 1 
for para in word_count.most_common(len(word_count)):    #распределеяем словарь в два списка (слова, значения) и считаем относительную частоту
    words.append(para[0])                                     
    count_abs.append(para[1])
    count_rel.append(para[1]/tokens_count)
    print(f'{n}.{para[0]} - {para[1]}')       #смотрим на результат: вывод слов и абсолютной частоты
    n += 1

Определяем части речи

In [ ]:
gramems = []

for word in words:
    p = pymorphy3_analyzer.parse(word)[0]
    tags = (str(p.tag)).split(',')
    gramems.append(tags[0])

Формируем датафрейм со всеми частями речи

In [ ]:
words_df = pd.DataFrame({'words' : words, 'gramems' : gramems, 'count' : count_abs, 'share' : count_rel})

In [ ]:
words_df.head()     #Посмотрим на начало датафрейма

### Фильтрация и запись в таблицу

В библиотеке есть свое обозначений частей речи (граммемы). По ним мы будем отфильтровывать только нужные нам слова. Полный список с примерами ниже:

| Граммема      | Значение                         | Пример         |
| :------------ | :------------------------------- | :------------- |
| NOUN          | Имя существительное              | Человек        |
| ADJF          | Имя прилагательное (полное)      | Хороший        |
| ADJS          | Имя прилагательное (краткое)     | Хорош          |
| COMP          | Компаратив                       | Быстрее, лучше |
| VERB          | Глагол (личная форма)            | Говорю         |
| INFN          | Глагол (инфинитив)               | Говорить       |
| PRTF          | Причастие (полное)               | Прочитанная    |
| PRTS          | Причастие (краткое)              | Прочитана      |
| GRND          | Деепричастие                     | Рассказывая    |
| NUMR          | Числительное                     | Три, сорок     |
| ADVB          | Наречие                          | Весело         |
| NPRO          | Местоимение-существительное      | Он, она        |
| PRED          | Предикатив (категория состояния) | Некогда        |
| PREP          | Предлог                          | Предлог        |
| CONJ          | Союз                             | И              |
| PRCL          | Частица                          | Бы, же, лишь   |
| INTJ          | Междометие                       | Ой             |
| LATN          | Латинские буквы                  | Hello          |
| PNCT          | Пунктуация                       | !?             |
| NUMB          | Число                            | 125            |
| ROMN          | Римское число                    | XI             |
| UNKN          | Не удалось разобрать             |                |



Подробнее тут: https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html

В переменную ниже впиши, какой фильтр тебе нужен. Возможные варианты:

* СУЩ (только существительные)
* ПРИЛ (только прилагательные, попадают и местоимения-прилагательные)
* ГЛАГ (только глаголы)
* НАРЕЧ (только наречия)
* СМЫСЛЫ (вместе сущ, прил, глаг и нареч)
* ДРУГОЕ (пользовательский фильтр, см. ниже)
* ВСЕ (все части речи, перечисленные в таблице выше)

In [ ]:
#впиши в кавычки нужный тебе вариант

grammems_filter = 'ДРУГОЕ'

In [ ]:
#если ты выбрал 'ДРУГОЕ', то можешь собрать свой фильтр из граммем из таблицы. Замени, убери или добавь значения в списке ниже

user_filter = ['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'ADVB', 'INTJ']

Теперь отфлитруем датафрейм по твоему запросу

In [ ]:
if grammems_filter == 'СУЩ':
    filtered_df = words_df[words_df.gramems == 'NOUN']
    
elif grammems_filter == 'ПРИЛ':
    filtered_df = words_df[words_df.gramems.isin(['ADJF', 'ADJS', 'COMP'])]
    
elif grammems_filter == 'ГЛАГ':
    filtered_df = words_df[words_df.gramems.isin(['VERB', 'INFN'])]
    
elif grammems_filter == 'НАРЕЧ':
    filtered_df = words_df[words_df.gramems == 'ADVB']
    
elif grammems_filter == 'СМЫСЛЫ':
    filtered_df = words_df[words_df.gramems.isin(['NOUN', 'ADJF', 'ADJS', 'COMP', 'VERB', 'INFN', 'ADVB'])]
    
elif grammems_filter == 'ДРУГОЕ':
    filtered_df = words_df[words_df.gramems.isin(user_filter)]
    
elif grammems_filter == 'ВСЕ':
    filtered_df = words_df
    
else:
    print('Похоже, такого фильтра не существует :(')

In [ ]:
#Посмотрим на начало отфильтрованного датафрейма

filtered_df.head()

Сохраняем датафрейм в файл формата xlsx

In [ ]:
#Впиши в кавычки ниже название для нового файла в формате 'Таблица.xlsx'

df_name = 'file_name.xlsx'

In [ ]:
filtered_df.to_excel(df_name)    #Сохраняем

Готово! Теперь у тебя есть данные, с которыми можно дальше работать. Удачи!

### Установка библиотек

Чтобы установить библиотеку, убери # в начале строки и запусти ячейку

In [ ]:
#!pip install pandas

In [ ]:
#!pip install python-docx

In [ ]:
#!pip install nltk

#import nltk
#nltk.download('stopwords')      # загрузка корпуса стоп-слов

In [ ]:
#!pip install emoji

In [ ]:
#!pip install pymorphy3